In [1]:
# py-sc-fermi

import timeit

start = timeit.default_timer()

from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.defect_charge_state import FrozenDefectChargeState
from py_sc_fermi.inputs import inputs_from_files

import numpy as np

In [2]:
example_dir = 'example_inputs'

unitcell_filename = f'{example_dir}/unitcell.dat'
totdos_filename = f'{example_dir}/totdos.dat'
input_fermi_filename = f'{example_dir}/input-fermi.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)

Volume of cell: 57.78888409009906 A^3
Found non-spin polarised system...
Number of electrons in system: 18
Energy gap of system: 0.8084 eV
Temperature: 100.0 K
Number of defect species: 2


In [3]:
defect_system = DefectSystem( defect_species=inputs['defect_species'],
                              volume=inputs['volume'],
                              nelect=inputs['nelect'],
                              edos=inputs['edos'],
                              dos=inputs['dos'],
                              egap=inputs['egap'], 
                              temperature=300.0 )

defect_system

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

In [4]:
from collections import Counter
nfrozen = 1
namef = ['V_Ga']
concf = [0.3285677364522e20]
# check frozen defects are included in the original list of defects
for name in namef:
    if name not in defect_system.defect_species_names:
        raise ValueError((f'ERROR: fixed concentration defect "{name}"'+
                           'is not in list of original defects!\n'+
                           'Please add new frozen defects as fixed charge defects in'
                           'a separate list - otherwise a solution cannot be found!!'))
# Check if any defects have been assigned multiple fixed concentrations
c = Counter(namef)
for n, count in c.items():
    if count > 1:
        raise ValueError(f'ERROR: defect "{n}"has multiple fixed concentrations!!')
for n, c in zip(namef, concf):
    defect_system.defect_species_by_name(n).fixed_concentration = c * defect_system.volume / 1e24
    
defect_system

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
fixed [c] = 0.0018987562837582401 cm^-3
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

In [5]:
ficharg = False
fixconch = False
namefn = ['V_Ga', 'Ga_i']
chargefn = [-1.0, 1.0]
concfn = [0.19e19, 0.5e20]
if 0 in chargefn:
    checkfixchg = True
else:
    checkfixchg = False
for n,q,c  in zip( namefn, chargefn, concfn ):
    print(c)
    c = c * defect_system.volume / 1e24
    frozen_charge_state = FrozenDefectChargeState(charge=q, concentration=c)
    if n in defect_system.defect_species_names:     
        defect_system.defect_species_by_name(n).charge_states[q] = frozen_charge_state
    else:
        ds = DefectSpecies(name=n, nsites=1, charge_states=[frozen_charge_state])
        defect_system.defect_species.append(ds)

defect_system.check_concentrations()
defect_system

1.9e+18
5e+19


DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
fixed [c] = 0.0018987562837582401 cm^-3
  q=+0.0, e=2.4451, deg=1
  q=-1.0, [c]=0.00010979887977118821
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

Ga_i, nsites=1
  q=+1.0, [c]=0.002889444204504953

In [6]:
defect_system.report()

SC Fermi level :      0.018315362596523502  (eV)

Concentrations:
n (electrons)  : 609633.7634268429 cm^-3
p (holes)      : 1.3479102448055966e+19 cm^-3
V_Ga           : 3.285677364522e+19 cm^-3
Ga_Sb          : 2.3737107494384077e-13 cm^-3
Ga_i           : 5e+19 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
V_Ga       : Charge Concentration(cm^-3) Total
           :  0.0  1.672432e+17          0.51 
           : -1.0  1.900000e+18          5.78  [fixed]
           : -2.0  3.078949e+19          93.71 
           : -3.0  4.159159e+13          0.00 
---------------------------------------------------------
Ga_Sb      : Charge Concentration(cm^-3) Total
           :  0.0  1.547224e-16          0.07 
           : -1.0  2.361934e-13          99.50 
           : -2.0  1.022993e-15          0.43 
---------------------------------------------------------
Ga_i       : Charge Concentration(cm^-3) Total
           :  1.0

/Users/bjm42/.pyenv/versions/3.7.0/lib/python3.7/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/defect_system.py:134: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/defect_system.py:131: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))


In [7]:
for ds in defect_system.defect_species:
    print(ds.name, ds.charge_state_concentrations(e_fermi=0.0, temperature=defect_system.temperature))
    for cs in ds.charge_states.values():
        print( cs.charge, cs.get_concentration(e_fermi=0.0, temperature=defect_system.temperature, nsite=ds.nsites))

V_Ga {0.0: 3.920052577743805e-05, -1.0: 0.00010979887977118821, -2.0: 0.0017497557143615952, -3.0: 1.163848018831286e-09}
0.0 8.397969256250837e-42
-1.0 0.00010979887977118821
-2.0 3.74851979756233e-40
-3.0 2.4933236703470463e-46
Ga_Sb {0.0: 8.94123733492443e-39, -1.0: 6.720905525451965e-36, -2.0: 1.4333371946423939e-38}
0.0 8.94123733492443e-39
-1.0 6.720905525451965e-36
-2.0 1.4333371946423939e-38
Ga_i {1.0: 0.002889444204504953}
1.0 0.002889444204504953


In [8]:
for ds in defect_system.defect_species:
    for cs in ds.charge_states.values():
        if not cs.fixed_concentration:
            print(cs.energy)

2.4451
2.3469
2.7146
2.2649
2.0937
2.2527


In [9]:
loweng = 2.65e-2
lowdeg = 1.0

In [10]:
import math

In [11]:
delphi = 0.0

for ds in defect_system.defect_species:
    sum1 = 0.0
    for cs in ds.charge_states.values():
        if cs.fixed_concentration:
            continue
        if cs.charge == 0:
            continue
        enerq = cs.energy + cs.charge * delphi - loweng
        print('enerq',enerq)
        expfac = -enerq / (kboltz * defect_system.temperature)
        print('expfac',expfac)
        sum1 += (cs.deg / lowdeg * math.exp(expfac))
        print(sum1)


enerq 2.3204000000000002


NameError: name 'kboltz' is not defined

In [ ]:
-2.230400000002/defect_system.kT

In [ ]:
math.exp(-258.8268797008544)

In [ ]:
from py_sc_fermi.constants import kboltz

In [ ]:
defect_system.temperature